In [1]:
import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
import itertools
from itertools import combinations_with_replacement, combinations, permutations

import bo_methods_lib
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
from bo_methods_lib.bo_methods_lib.analyze_data import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import * #Fix this later
import pympler
import pickle
import signac

from pympler import asizeof

from matplotlib import pyplot as plt

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200

#Ignore inconcistent version warning
import warnings
from sklearn.exceptions import InconsistentVersionWarning
warnings.filterwarnings(action='ignore', category=InconsistentVersionWarning)

In [2]:
#From signac
import signac
meth_name_val_list = [1, 2, 3, 4, 5, 6]
save_csv = False

criteria_dict = {"cs_name_val" : 1,
                 "ep_enum_val": {"$gte": 1},
                 "meth_name_val": {"$in": meth_name_val_list}}

criteria_dict = {"cs_name_val" : 1,
                 "ep_enum_val": 1,
                 "meth_name_val": {"$in": meth_name_val_list}}

# criteria_dict = {"cs_name_val" : 1,
#                  "param_name_str" : "t1t2",
#                  "ep_enum_val":1,
#                  "retrain_GP": 25,
#                  "num_x_data": 5,
#                  "outputscl": None,
#                  "bo_iter_tot": 50,
#                  "lenscl": None}


In [8]:
project = signac.get_project()
save_csv = False
analyzer = General_Analysis(criteria_dict, project, save_csv)
print(analyzer.make_dir_name_from_criteria(criteria_dict))

# df_all_jobs, job_list, theta_true = analyzer.get_df_all_jobs()
# print(min(df_all_jobs["Min Obj Act"]))
# print(df_all_jobs["BO Method"].unique())

df_best, job_list_best = analyzer.get_best_data()

# for i in range(len(job_list_best)): 
#     print(job_list_best[i].id)
    
print(job_list_best)

Results\cs_name_val_1/ep_enum_val_1/meth_name_val_in_1_2_3_4_5_6
[Job(project=Project('c:\\Users\\mcarlozo\\Documents\\Repos\\Toy_Problem'), statepoint={'cs_name_val': 1, 'meth_name_val': 1, 'param_name_str': 't1t2', 'ep0': 1, 'ep_enum_val': 1, 'sep_fact': 1.0, 'normalize': True, 'gen_heat_map_data': True, 'noise_mean': 0, 'noise_std': 0.01, 'kernel_enum_val': 1, 'lenscl': None, 'outputscl': None, 'retrain_GP': 25, 'reoptimize_obj': 20, 'bo_iter_tot': 50, 'bo_run_tot': 5, 'save_data': False, 'seed': 1, 'ei_tol': 1e-07, 'obj_tol': 1e-07, 'num_x_data': 5, 'gen_meth_theta': 1, 'gen_meth_x': 2, 'gen_meth_theta_val': 2, 'num_theta_multiplier': 10, 'num_val_pts': 20}), Job(project=Project('c:\\Users\\mcarlozo\\Documents\\Repos\\Toy_Problem'), statepoint={'cs_name_val': 1, 'meth_name_val': 2, 'param_name_str': 't1t2', 'ep0': 1, 'ep_enum_val': 1, 'sep_fact': 1.0, 'normalize': True, 'gen_heat_map_data': True, 'noise_mean': 0, 'noise_std': 0.01, 'kernel_enum_val': 1, 'lenscl': None, 'outputscl':

In [11]:
df_best
# print(job_list_best[0].fn("signac_statepoint.json"))

,Run Number,BO Iter,Best Error,Exploration Bias,Max EI,Theta Max EI,Min Obj,Min Obj Act,Theta Min Obj,Min Obj Cum.,...,Regret,Speed,Time/Iter,EP Method Val,BO Method,Job ID,Max Evals,Termination,Total Run Time,L2 Norm Theta
372,1,8,9.029940e-02,1.0,0.000000e+00,"[0.3157894736842106, -0.5263157894736843]",6.610628e-02,1.354291e-02,"[0.9715742681315809, -1.0126754621409975]",0.013543,...,0.001223,0.000194,2.373189,CONSTANT,A1,7d335fac3f3dbef1def51beac296d8bf,23,ei-obj,54.583346,0.031124
818,3,7,-1.892925e+00,1.0,0.000000e+00,"[-0.3157894736842106, 1.3684210526315788]",-2.303043e+00,-3.606177e+00,"[0.9529348098348527, -1.0121318009896898]",-3.606177,...,0.000007,0.003854,3.230876,CONSTANT,B1,f77eb15458c0df1814cb2d097bc16619,7,ei-regret,22.616132,0.048604
649,1,42,1.396315e-07,1.0,2.263683e-74,"[0.3157894736842106, -0.5263157894736843]",9.457331e-08,7.463492e-08,"[0.9999248708378773, -1.000023130712286]",0.0,...,0.000004,0.001994,14.762938,CONSTANT,A2,bfaeb98a04d2a186bf9fc0c5a87602e7,43,obj,634.806335,0.000079
250,2,40,-2.040383e+01,1.0,8.980287e-03,"[0.9999882821330597, -1.0000087991244813]",-1.769546e+01,-1.891750e+01,"[1.0000132749127315, -1.0000112751811214]",-18.917498,...,77.330711,1.787861,92.297088,CONSTANT,B2,6c8f446e38438959407a256119512c05,50,max_budget,4614.854407,0.000017
121,4,20,8.565752e-05,1.0,2.978017e-07,"[0.8603040674988498, -0.910657604164606]",9.371427e-07,1.260755e-06,"[1.0001155298667694, -0.9998179138007688]",0.000001,...,0.000014,0.000383,11.821221,CONSTANT,C2,08978be5acb14bc5c527c23532eaa621,35,obj-regret,413.742731,0.000216
437,1,19,7.555456e-05,1.0,2.116127e-07,"[0.9323396570779297, -1.048030891108625]",5.692213e-07,2.295503e-07,"[0.9999769329516789, -0.9999187904922076]",0.0,...,0.000008,0.000194,15.449586,CONSTANT,D2,811c3b5e401ba0adcedd67a67dbf2457,34,obj-regret,525.285929,0.000084


In [ ]:
run_num_list = list(map(int, df_best["Run Number"].to_numpy()))
bo_iter_list = list(map(int, df_best["BO Iter"].to_numpy()))
bo_method_list = list(df_best["BO Method"].to_numpy())

In [ ]:
z_choices = ["min_sse", "sse"]

x_label = "BO Iterations"
y_label = r"$\mathbf{e(\theta)}$"
# y_label = "Max " + r"$\mathbf{EI(\theta)}$"
save_path = make_dir_name_from_criteria(criteria_dict)
plot_dict = make_plot_dict(False, None, x_label, None, line_levels = None, save_path=save_path)

file_path_list = [job.fn("BO_Results.gz") for job in job_list_best]

plot_objs_all_methods(file_path_list, run_num_list, z_choices, plot_dict)

In [ ]:
z_choice = "min_sse"

x_label = "BO Iterations"
# y_label = r"$\mathbf{e(\theta)}$"
y_label = "Max " + r"$\mathbf{EI(\theta)}$"
# title = "Min SSE Values"
save_path = make_dir_name_from_criteria(criteria_dict)
plot_dict = make_plot_dict(False, None, x_label, y_label, line_levels = None, save_path=save_path)

file_path_list = [job.fn("BO_Results.gz") for job in job_list]

plot_one_obj_all_methods(file_path_list, run_num_list, z_choice, plot_dict)

In [ ]:
pair = 0
z_choice = "sse_mean"
# title = "Heat Map Pair " + z_choice
levels = [100,100,100, 100, 100, 100]
save_path = make_dir_name_from_criteria(criteria_dict)
plot_dict = make_plot_dict(False, None, None, None, line_levels = levels, save_path=save_path)
file_path_list = [job.fn("BO_Results.gz") for job in job_list_best]

plot_hms_all_methods(file_path_list, run_num_list, bo_iter_list, pair, z_choice, plot_dict)

In [ ]:
#Make Heat maps for all pairs
z_choices = ["sse_sim", "sse_mean", "sse_var", "ei"]
levels = [100,100,100,100]

#Loop over best jobs
for i in range(len(job_list_best)):   
    #Set file path
    file_path = job_list_best[i].fn("BO_Results.gz")
    #Back out number of parameters
    string_val = df_best["Theta Min Obj"].iloc[0]
    try:
        numbers = [float(num) for num in string_val.replace('[', '').replace(']', '').split()]
    except:
        numbers = [float(num) for num in string_val]
        
    #Create list of parameter pair combinations
    dim_theta = len(np.array(numbers).reshape(-1, 1))
    dim_list = np.linspace(0, dim_theta-1, dim_theta)
    pairs = len((list(combinations(dim_list, 2))))
    
    #Loop over parameter pairs
    for pair in range(pairs):
        #Set save path
        save_path = make_dir_name_from_criteria(criteria_dict)
        plot_dict = make_plot_dict(False, None, None, None, line_levels = levels, save_path=save_path)
        plot_hms_gp_compare(file_path, run_num_list[i], bo_iter_list[i], pair, z_choices, plot_dict)

In [ ]:
#Get HP Plots
#Set parameters
file_path = "2023/09/05/Data_Files/CS2_BO_method_C2_ep_method_BOYLE.pickle"
file_path = "2023/09/05/Data_Files/CS2_BO_method_B1_ep_method_CONSTANT.pickle"
file_path = "workspace/811c3b5e401ba0adcedd67a67dbf2457/BO_Results.gz"

x_label = "BO Iterations"
y_label = "Value"
title = "Hyperparameter Values"

save_path = os.path.join(os.path.dirname(file_path), "line_plots", "hyperparams")
plot_dict = make_plot_dict(False, title, x_label, y_label, ybins = 7, line_levels = None, save_path=save_path)

hps, hp_names, hp_true = analyze_hypers(file_path, save_csv = True)
plot_2D_Data_w_BO_Iter(hps, hp_names, hp_true, plot_dict)

In [ ]:
#Get sse, log_sse, and ei plots
file_path = "workspace/811c3b5e401ba0adcedd67a67dbf2457/BO_Results.gz" #CS1 2D Best

z_choices = ["sse", "min_sse", "ei"]
z_choices_sort = sorted(z_choices, key=lambda x: ("sse", "min_sse", "ei").index(x))

x_label = "BO Iterations"
y_label = "Values"
title = None

save_path = os.path.join(os.path.dirname(file_path), "line_plots", '_'.join(map(str, z_choices_sort)))
plot_dict = make_plot_dict(False, title, x_label, y_label, ybins = 7, line_levels = None, save_path=save_path)

data, data_names, data_true, GP_meth_val = analyze_sse_min_sse_ei(file_path, z_choices)
plot_2D_Data_w_BO_Iter(data, data_names, data_true, plot_dict)

In [ ]:
#Get theta @ min sse
file_path = "2023/08/28/Data_Files/CS1_meth_A2_sep_fact_1.0.pickle"
file_path = "2023/09/05/Data_Files/CS2_BO_method_C2_ep_method_BOYLE.pickle"
file_path = "2023/09/05/Data_Files/CS2_BO_method_B1_ep_method_CONSTANT.pickle"
file_path = "workspace/811c3b5e401ba0adcedd67a67dbf2457/BO_Results.gz"

z_choice = "sse"

x_label = "BO Iterations"
title = "Min Obj Parameter Values"

save_path = os.path.join(os.path.dirname(file_path), "line_plots", z_choice + "_params")
plot_dict = make_plot_dict(False, title, x_label, None, line_levels = None, save_path=save_path)

data, data_names, data_true = analyze_thetas(file_path, z_choice)
plot_2D_Data_w_BO_Iter(data, data_names, data_true, plot_dict)

In [ ]:
#Get theta @ min sse overall
file_path = "workspace/811c3b5e401ba0adcedd67a67dbf2457/BO_Results.gz"

z_choice = "min_sse"

x_label = "BO Iterations"
title = "Min Obj Parameter Values Overall"

save_path = os.path.join(os.path.dirname(file_path), "line_plots", z_choice + "_params")
plot_dict = make_plot_dict(False, title, x_label, None, line_levels = None, save_path=save_path)

data, data_names, data_true = analyze_thetas(file_path, z_choice)
plot_2D_Data_w_BO_Iter(data, data_names, data_true, plot_dict)

In [ ]:
#Get theta @ Max EI
file_path = "2023/08/28/Data_Files/CS1_meth_A2_sep_fact_1.0.pickle"
file_path = "2023/09/05/Data_Files/CS2_BO_method_C2_ep_method_BOYLE.pickle"
file_path = "2023/09/05/Data_Files/CS2_BO_method_B1_ep_method_CONSTANT.pickle"
file_path = "workspace/811c3b5e401ba0adcedd67a67dbf2457/BO_Results.gz"

z_choice = "ei"

x_label = "BO Iterations"
y_label = "Theta Values"
title = "Max EI Parameter Values"

save_path = os.path.join(os.path.dirname(file_path), "line_plots", z_choice + "_params")
plot_dict = make_plot_dict(False, title, x_label, y_label, line_levels = None, save_path=save_path)

data, data_names, data_true = analyze_thetas(file_path, z_choice)
plot_2D_Data_w_BO_Iter(data, data_names, data_true, plot_dict)

In [ ]:
# #Get Median Data from sf experiment
df_median, job_list_med = get_median_data(criteria_dict, df, job_list, theta_true, save_csv)

for i in range(len(job_list_med)): 
    print(job_list_med[i].id)
    
df_median

In [ ]:
# #Get Mean Data from sf experiment
df_mean, job_list_mean = get_mean_data(criteria_dict, df, job_list, theta_true, save_csv)

for i in range(len(job_list_mean)): 
    print(job_list_mean[i].id)
    
df_mean